In [21]:
import pandas as pd

In [56]:
cultivars = pd.read_csv("Cultivars.csv")
cultivars.head()

,Cultivar Name,Cultivar ID,Subpopulation,Location,Accession
0,02428,C152,Temperate Japonica,Jiangsu,SRR1239752
1,02428-IL,CX277,Temperate Japonica,China,ERS470588
2,053A-3,CX351,Temperate Japonica,China,ERS470636
3,10340,W146,Indica I,Italy,SRR1239949
4,104_UPLA,IRIS_313-8063,Tropical Japonica,Argentina,ERS468624


In [57]:
heights = pd.read_csv("Height.csv")
heights.head()

,Cultivar Name,Cultivar ID,Subpopulation,Plant Height (cm)
0,02428,C152,Temperate Japonica,100.0
1,10340,W146,Indica I,127.4
2,108S,W150,Indica II,104.6
3,2,W129,VI/Aromatic,151.2
4,4484,W086,Indica II,103.4


In [58]:
genotype = pd.read_csv("Genotype.csv")
genotype = genotype.rename(columns={'Variation ID': 'Cultivar ID'})
genotype.head()

,Cultivar ID,vg0130976864,vg0135617816,vg0138428840,vg0405463422,vg0713178880,vg0819793460,vg1019044175,vg1123563633,vg1207667840
0,Chromosome,chr01,chr01,chr01,chr04,chr07,chr08,chr10,chr11,chr12
1,Position,30976864,35617816,38428840,5463422,13178880,19793460,19044175,23563633,7667840
2,Variation Type,SNP,SNP,SNP,SNP,SNP,SNP,SNP,SNP,SNP
3,Reference,A,C,T,G,T,A,C,T,T
4,Primary Allele,G,C,T,G,T,A,T,T,T


In [59]:
genotype.shape

(4732, 10)

In [60]:
# counting del values for each SNP

delCount = []
for i in range(1, genotype.shape[1]):
    val = 0
    for j in range(6, genotype.shape[0]):
        if genotype.iloc[j, i] == "DEL":
            val = val + 1
    delCount.append((genotype.columns[i],val))

print(sorted(delCount, key=lambda x : x[1], reverse=True))

[('vg0405463422', 1667), ('vg1123563633', 384), ('vg0135617816', 148), ('vg0130976864', 0), ('vg0138428840', 0), ('vg0713178880', 0), ('vg0819793460', 0), ('vg1019044175', 0), ('vg1207667840', 0)]


In [61]:
# counting n values for each SNP

delCount = []
for i in range(1, genotype.shape[1]):
    val = 0
    for j in range(6, genotype.shape[0]):
        if genotype.iloc[j, i] == "N":
            val = val + 1
    delCount.append((genotype.columns[i],val))

print(sorted(delCount, key=lambda x : x[1], reverse=True))

[('vg0405463422', 815), ('vg0130976864', 79), ('vg0135617816', 64), ('vg0138428840', 17), ('vg0819793460', 11), ('vg1123563633', 9), ('vg1207667840', 7), ('vg0713178880', 4), ('vg1019044175', 1)]


In [62]:
# vg04 has very high frequency of del, hence we drop it

genotype.drop("vg0405463422", axis=1, inplace=True)

In [63]:
genotypeHeader = genotype.iloc[:6, :]
primaryAlleleRow = 4
secondaryAlleleRow = 5
genotypeHeader

,Cultivar ID,vg0130976864,vg0135617816,vg0138428840,vg0713178880,vg0819793460,vg1019044175,vg1123563633,vg1207667840
0,Chromosome,chr01,chr01,chr01,chr07,chr08,chr10,chr11,chr12
1,Position,30976864,35617816,38428840,13178880,19793460,19044175,23563633,7667840
2,Variation Type,SNP,SNP,SNP,SNP,SNP,SNP,SNP,SNP
3,Reference,A,C,T,T,A,C,T,T
4,Primary Allele,G,C,T,T,A,T,T,T
5,Secondary Allele,A,T,C,A,T,C,C,C


In [64]:
genotype = genotype.iloc[6:, :]
genotype

,Cultivar ID,vg0130976864,vg0135617816,vg0138428840,vg0713178880,vg0819793460,vg1019044175,vg1123563633,vg1207667840
6,B001 (Heibiao) Imputed,G,C,T,A,A,C,T,T
7,B002 (Sansuijin) Imputed,A,C,T,T,A,C,T,T
8,B003 (Zaoshengbai__) Imputed,A,C,T,T,A,C,T,T
9,B004 (Qiuguangtengxi_104_) Imputed,A,C,T,T,A,C,T,T
10,B005 (Wanshi) Imputed,A,C,T,A,A,C,DEL,T
...,...,...,...,...,...,...,...,...,...
4727,W326 (Haolvguangnian) Imputed,G,C,T,T,A,C,T,C
4728,W327 (JinghuB) Imputed,A,C,T,T,A,C,T,C
4729,W328 (Sibeitichao6) Imputed,A,C,T,T,A,C,T,T
4730,W329 (GPNO_5055) Imputed,G,C,T,T,A,T,T,C


In [65]:
col = genotype.iloc[:, 4]
print(type(col))

<class 'pandas.core.series.Series'>


In [66]:
# replacing genotype for each column with their primary allele

for i in range(1, genotype.shape[1]):
    primaryAllele = genotypeHeader.iloc[primaryAlleleRow, i]
    secondaryAllele = genotypeHeader.iloc[secondaryAlleleRow, i]
    col = genotype.iloc[:, i]
    col.replace("DEL", secondaryAllele, inplace=True)
    col.replace("N", primaryAllele, inplace=True)

genotype

,Cultivar ID,vg0130976864,vg0135617816,vg0138428840,vg0713178880,vg0819793460,vg1019044175,vg1123563633,vg1207667840
6,B001 (Heibiao) Imputed,G,C,T,A,A,C,T,T
7,B002 (Sansuijin) Imputed,A,C,T,T,A,C,T,T
8,B003 (Zaoshengbai__) Imputed,A,C,T,T,A,C,T,T
9,B004 (Qiuguangtengxi_104_) Imputed,A,C,T,T,A,C,T,T
10,B005 (Wanshi) Imputed,A,C,T,A,A,C,C,T
...,...,...,...,...,...,...,...,...,...
4727,W326 (Haolvguangnian) Imputed,G,C,T,T,A,C,T,C
4728,W327 (JinghuB) Imputed,A,C,T,T,A,C,T,C
4729,W328 (Sibeitichao6) Imputed,A,C,T,T,A,C,T,T
4730,W329 (GPNO_5055) Imputed,G,C,T,T,A,T,T,C


In [67]:
def extractVariantID(cultivarID):
    return cultivarID.split(sep=" ")[0]

genotype['Cultivar ID'] = genotype['Cultivar ID'].map(extractVariantID)
genotype

,Cultivar ID,vg0130976864,vg0135617816,vg0138428840,vg0713178880,vg0819793460,vg1019044175,vg1123563633,vg1207667840
6,B001,G,C,T,A,A,C,T,T
7,B002,A,C,T,T,A,C,T,T
8,B003,A,C,T,T,A,C,T,T
9,B004,A,C,T,T,A,C,T,T
10,B005,A,C,T,A,A,C,C,T
...,...,...,...,...,...,...,...,...,...
4727,W326,G,C,T,T,A,C,T,C
4728,W327,A,C,T,T,A,C,T,C
4729,W328,A,C,T,T,A,C,T,T
4730,W329,G,C,T,T,A,T,T,C


In [68]:
mapping = pd.merge(genotype, heights.iloc[:, 1:4], on='Cultivar ID' )

In [69]:
mapping

,Cultivar ID,vg0130976864,vg0135617816,vg0138428840,vg0713178880,vg0819793460,vg1019044175,vg1123563633,vg1207667840,Subpopulation,Plant Height (cm)
0,C001,G,C,T,T,T,C,C,C,Indica I,144.13
1,C002,G,C,C,T,T,C,C,C,VI/Aromatic,177.62
2,C003,G,C,T,T,A,C,T,T,Japonica Intermediate,141.57
3,C004,G,C,T,A,A,C,T,C,Temperate Japonica,140.40
4,C005,G,C,T,A,A,T,T,C,Tropical Japonica,163.33
...,...,...,...,...,...,...,...,...,...,...,...
524,W326,G,C,T,T,A,C,T,C,Japonica Intermediate,155.40
525,W327,A,C,T,T,A,C,T,C,Temperate Japonica,89.80
526,W328,A,C,T,T,A,C,T,T,Intermediate,117.40
527,W329,G,C,T,T,A,T,T,C,Tropical Japonica,147.40


In [70]:
mapping.to_csv('mapping.csv', index=False)

In [71]:
mapping['sequence'] = mapping.apply(lambda x: ''.join(x[1:9]), axis=1)

In [72]:
mapping

,Cultivar ID,vg0130976864,vg0135617816,vg0138428840,vg0713178880,vg0819793460,vg1019044175,vg1123563633,vg1207667840,Subpopulation,Plant Height (cm),sequence
0,C001,G,C,T,T,T,C,C,C,Indica I,144.13,GCTTTCCC
1,C002,G,C,C,T,T,C,C,C,VI/Aromatic,177.62,GCCTTCCC
2,C003,G,C,T,T,A,C,T,T,Japonica Intermediate,141.57,GCTTACTT
3,C004,G,C,T,A,A,C,T,C,Temperate Japonica,140.40,GCTAACTC
4,C005,G,C,T,A,A,T,T,C,Tropical Japonica,163.33,GCTAATTC
...,...,...,...,...,...,...,...,...,...,...,...,...
524,W326,G,C,T,T,A,C,T,C,Japonica Intermediate,155.40,GCTTACTC
525,W327,A,C,T,T,A,C,T,C,Temperate Japonica,89.80,ACTTACTC
526,W328,A,C,T,T,A,C,T,T,Intermediate,117.40,ACTTACTT
527,W329,G,C,T,T,A,T,T,C,Tropical Japonica,147.40,GCTTATTC


In [73]:
len(mapping['sequence'].unique())

52